In [1]:
#Script to update related child subject identifier on Mother's schedule

In [ ]:
# Imports
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
from edc_base.utils import formatted_age, get_utcnow
from dateutil.relativedelta import relativedelta
from edc_appointment.models import Appointment

In [2]:
for a in OnScheduleCohortAAntenatal.objects.all():
    try:
        cb = ChildBirth.objects.get(subject_identifier__startswith=a.subject_identifier)
    except ChildBirth.DoesNotExist:
        pass
    else:
        a.child_subject_identifier = cb.subject_identifier
        a.save()

In [3]:

def update_child_pids(sid):
    

    schedules = []
    schedule_names = []
    for a in Appointment.objects.filter(subject_identifier=sid):
        if [a.schedule.onschedule_model, a.schedule_name, a.subject_identifier] not in schedules:
            schedules.append([a.schedule.onschedule_model, a.schedule_name, a.subject_identifier])
    
    child_subject_identifier = None
    
            
    for sch, sch_n, subject_identifier in schedules:
    
        if 'enrol' in sch_n or 'antenatal' in sch_n or ('sec' in sch_n and 'quart' not in sch_n):
            onschedule_model = django_apps.get_model(sch)
            try:
                onschedule_obj = onschedule_model.objects.get(
                    subject_identifier=subject_identifier,
                    schedule_name=sch_n)
            except onschedule_model.DoesNotExist:
                print("\n\n??????", sch_n, subject_identifier, sch)
            else:
                child_subject_identifier = onschedule_obj.child_subject_identifier
    
    if child_subject_identifier and len(child_subject_identifier.split('-')) ==4:
        for sch, sch_n, subject_identifier in schedules:
    
            if 'enrol' not in sch_n or 'antenatal' not in sch_n or 'birth' not in sch_n :
                onschedule_model = django_apps.get_model(sch)
                try:
                    onschedule_obj = onschedule_model.objects.get(
                        subject_identifier=subject_identifier,
                        schedule_name=sch_n)
                except onschedule_model.DoesNotExist:
                    print("\n\n??????", sch_n, subject_identifier, sch)
                else:
                    onschedule_obj.child_subject_identifier = child_subject_identifier
                    onschedule_obj.save()
       

            


In [4]:
sids = list(set(SubjectConsent.objects.all().values_list('subject_identifier', flat=True)))
for sid in sids:
    update_child_pids(sid)



?????? b_sec_quart1_schedule1 B142-040990006-4 flourish_caregiver.onschedulecohortbsecquart


?????? b_sec_quart1_schedule1 B142-040990030-4 flourish_caregiver.onschedulecohortbsecquart


?????? b_sec_quart1_schedule1 B142-040990010-6 flourish_caregiver.onschedulecohortbsecquart


?????? b_sec_quart1_schedule1 B142-040990009-8 flourish_caregiver.onschedulecohortbsecquart


?????? b_sec_quart1_schedule1 B142-040990053-6 flourish_caregiver.onschedulecohortbsecquart


?????? a_enrol2_schedule1 B142-040990558-4 flourish_caregiver.onschedulecohortaenrollment


?????? a_enrol2_schedule1 B142-040990558-4 flourish_caregiver.onschedulecohortaenrollment


?????? c_sec_quart1_schedule1 B142-040990004-9 flourish_caregiver.onschedulecohortcsecquart


?????? c_sec2_schedule1 B142-040990099-9 flourish_caregiver.onschedulecohortcsec


?????? c_sec2_schedule1 B142-040990099-9 flourish_caregiver.onschedulecohortcsec


In [7]:
update_child_pids('B142-040990014-8')


>>>>>>> b_enrol1_schedule1
>>>>>>> b_quarterly1_schedule1
>>>>>>> b_fu_quarterly1_schedule1
>>>>>>> b_fu1_schedule1
